In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI Reduction服务器的PyTorch分布式训练

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/reduction_server/pytorch_distributed_training_reduction_server.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

当您在使用GPU跨多个节点运行分布式训练作业时，节点之间的梯度通信可能会导致显着的延迟。Reduction Server是一种全局约简算法，可以提高分布式训练的吞吐量并减少延迟。本笔记本演示了如何在Vertex AI上运行带有Reduction Server的PyTorch分布式训练作业。训练作业旨在对来自Hugging Face Transformers库的预训练模型`bert-large-cased`在`imdb`数据集上进行情感分类进行微调。

了解有关[Vertex AI训练](https://cloud.google.com/vertex-ai/docs/training/custom-training)和[Vertex AI Reduction Server](https://cloud.google.com/blog/topics/developers-practitioners/optimize-training-performance-reduction-server-vertex-ai)的更多信息。

### 目标

在这个笔记本中，您将学习如何创建一个使用PyTorch分布式训练框架和工具的PyTorch分布式训练作业，并在Vertex AI训练服务上使用Reduction Server运行训练作业。

本教程使用以下Google Cloud ML服务和资源：

* Vertex AI训练
* Cloud Storage

执行的步骤包括：

* 创建一个PyTorch分布式训练应用程序
* 将训练应用程序打包到预构建的容器中
* 在Vertex AI上创建一个带有Reduction Server的自定义作业
* 提交并监控作业.

### 数据集

在本教程中，我们使用了来自Hugging Face的[`imdb`](https://huggingface.co/datasets/imdb)数据集。`imdb`是一个用于二元情感分类的大型电影评论数据集，包含了用于训练的25000条极性强烈的电影评论和25000条用于测试的评论。

成本

本教程使用Google Cloud的可计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI收费](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage收费](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预计使用量生成成本估算。

## 安装

安装以下所需的软件包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform 

### 仅限Colab使用：取消下面的单元格注释以重新启动内核。

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 在开始之前

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您可获得 $300 的免费信用用于计算/存储成本。

2. [确保为您的项目启用了计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用以下 API：Vertex AI API，Cloud Resource Manager API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,cloudresourcemanager.googleapis.com)。

4. 如果您在本地运行此笔记本，您需要安装 [Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
- 运行 `gcloud config list`。
- 运行 `gcloud projects list`。
- 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改Vertex AI使用的“REGION”变量。了解有关[Vertex AI区域](https://cloud.google.com/vertex-ai/docs/general/locations)的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行现场教程会话，则可能正在使用共享的测试帐户或项目。为了避免在创建的资源上出现用户之间的名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源名称上。

In [ ]:
import random
import string


# Generate a uuid of a specified length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 认证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行认证。请按照以下相关说明进行操作。

**1. Vertex AI 工作台**
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请查看如何在 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 上为您的服务账户授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间工件，如数据集。

当您使用 Cloud SDK 提交一个训练作业时，您会将包含训练代码的 Python 包上传到一个云存储桶。Vertex AI 会从这个包中运行代码。在本教程中，Vertex AI 还会将作业产生的经过训练的模型保存在同一个存储桶中。使用这个模型工件，然后您可以创建 Vertex AI 模型资源并用于预测。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

### 导入库

In [ ]:
from google.cloud import aiplatform

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

# 教程

现在您已经准备好开始创建PyTorch分布式训练作业。

### 打包训练应用程序

在在 Vertex AI 上运行训练作业之前，必须将训练应用程序代码和任何依赖项打包并上传到您的谷歌云项目可以访问的云存储桶或容器注册表或 Artifact Registry 中。本节将展示如何在云中打包和部署您的应用程序。

有两种方式可以打包您的应用程序和依赖项，并在 Vertex AI 上进行训练：

1. 使用训练代码和依赖项创建 [Python 源分发](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container)，然后与 Vertex AI 上的[预构建容器](https://cloud.google.com/vertex-ai/docs/training/pre-built-containers)一起使用
2. 使用 [自定义容器](https://cloud.google.com/ai-platform/training/docs/custom-containers-training) 使用 Docker 容器打包依赖项

**本笔记展示了使用 Python 源分发选项在 Vertex AI 上运行自定义训练作业。**

推荐的培训应用程序结构

您可以按照自己喜欢的方式构建培训应用程序。然而，[以下结构](https://cloud.google.com/vertex-ai/docs/training/create-python-pre-built-container#structure)通常在 Vertex AI 示例中使用，并且使您的项目组织类似于示例可以让您更容易跟随示例。

```
.
├── python_package
│   ├── README.md
│   ├── setup.py
│   └── trainer
│       ├── __init__.py
│       └── task.py
└── pytorch-distributed-training-reduction-server.ipynb    --> 这个笔记本
```

1. 主项目目录包含 `setup.py` 文件与依赖关系。
2. 使用名为 `trainer` 的子目录存储您的主应用程序模块和 `scripts` 以在本地或云中提交培训作业。
3. 在 `trainer` 目录内：
    - `task.py` - 主应用程序模块 1) 初始化 PyTorch 分布式培训环境，以及 2) 运行模型培训和评估实验，并导出最终模型。
    - `__init__.py` 是必需的，使Python将包含该文件的目录视为包。

为训练应用程序定义变量

初始化变量以定义预构建容器映像、训练应用程序的位置和训练模块。

In [ ]:
APP_NAME = "pytorch-bert"

PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-9:latest"
)

PYTHON_PACKAGE_APPLICATION_DIR = "python_package"

source_package_file_name = f"{PYTHON_PACKAGE_APPLICATION_DIR}/dist/trainer-0.1.tar.gz"
python_package_gcs_uri = (
    f"{BUCKET_URI}/pytorch-on-gcp/{APP_NAME}/train/python_package/trainer-0.1.tar.gz"
)

python_module_name = "trainer.task"

创建培训应用程序的文件结构

In [ ]:
! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/README.md

! mkdir {PYTHON_PACKAGE_APPLICATION_DIR}/trainer
! touch {PYTHON_PACKAGE_APPLICATION_DIR}/trainer/__init__.py

创建用于训练应用程序的 `setup.py` 文件

以下是用于训练应用程序的 `setup.py` 文件。`setup.py` 中的 `find_packages()` 函数包含了 `trainer` 目录，因为该目录包含了 `__init__.py` 文件，它告诉 [Python Setuptools](https://setuptools.readthedocs.io/en/latest/) 将父目录的所有子目录作为依赖项包含进来。

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/setup.py

import os
from setuptools import find_packages
from setuptools import setup
import setuptools

from distutils.command.build import build as _build
import subprocess


REQUIRED_PACKAGES = [
    'transformers==4.28.0',
    'datasets',
    'evaluate',
]

setup(
    name='trainer',
    version='0.1',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='Vertex AI | Training | PyTorch | Text Classification | Python Package'
)


创建训练应用程序代码

`task.py` 是主要的应用程序模块。它初始化 PyTorch 分布式训练环境，并运行模型训练和评估实验，并导出最终模型。

In [ ]:
%%writefile ./{PYTHON_PACKAGE_APPLICATION_DIR}/trainer/task.py
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the \"License\");
# you may not use this file except in compliance with the License.\n",
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an \"AS IS\" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import numpy as np
import pandas as pd
from datetime import datetime
import argparse

import torch
import torch.distributed as dist
torch.cuda.empty_cache()

import datasets
from datasets import ClassLabel, Sequence, load_dataset

import transformers
from transformers import (
    AutoModelForSequenceClassification, 
    AutoTokenizer,
    EvalPrediction, 
    Trainer, 
    TrainingArguments,
    default_data_collator)

from google.cloud import storage


def main():

  parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
  parser.add_argument("--epochs", type=int, help="Number of training epochs.", default=2)
  parser.add_argument("--batch_size", type=int, help="Training batch size for one process.", default=32)
  parser.add_argument("--model_dir", type=str, help="Directory for saving models.", default=os.environ['AIP_MODEL_DIR'] if 'AIP_MODEL_DIR' in os.environ else "")
  argv = parser.parse_args()

  model_name_or_path = "bert-large-uncased"
  padding = "max_length"
  max_seq_length = 128

  datasets = load_dataset("imdb")
  label_list = datasets["train"].unique("label")
  label_to_id = {1: 1, 0: 0, -1: 0}

  tokenizer = AutoTokenizer.from_pretrained(
      model_name_or_path,
      use_fast=True,
  )

  def preprocess_function(examples):
      """
      Tokenize the input example texts
      """
      args = (examples["text"],)
      result = tokenizer(
          *args, padding=padding, max_length=max_seq_length, truncation=True
      )

      # Map labels to IDs (not necessary for GLUE tasks)
      if label_to_id is not None and "label" in examples:
          result["label"] = [label_to_id[example] for example in examples["label"]]

      return result

  # apply preprocessing function to input examples
  datasets = datasets.map(preprocess_function, batched=True, load_from_cache_file=True)

  model = AutoModelForSequenceClassification.from_pretrained(
      model_name_or_path, 
      num_labels=len(label_list)
  )

  ngpus_per_node = torch.cuda.device_count()
  world_size = int(os.environ["WORLD_SIZE"])
  
  # Since we have ngpus_per_node processes per node, the total world_size
  # needs to be adjusted accordingly
  world_size =  world_size * ngpus_per_node

  start = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  print(f'Starting distributed training: {start}') 
  
  # Use torch.multiprocessing.spawn to launch distributed processes
  torch.multiprocessing.spawn(main_worker,
    args = (ngpus_per_node, world_size, datasets, model, tokenizer, argv),
    nprocs = ngpus_per_node,
    join = True)
  
  end = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
  print(f'Distributed training complete: {end}')

def main_worker(local_rank, ngpus_per_node, world_size, datasets, model, tokenizer, argv):

  # This is the (global) rank of the current process
  rank = int(os.environ["RANK"])
  
  # For multiprocessing distributed training, rank needs to be the
  # global rank among all the processes
  rank = rank * ngpus_per_node + local_rank
  print (f"Distributed and Multi-processing. Setting rank for each worker. rank={rank}")

  dist.init_process_group(
      backend="nccl", 
      init_method="env://",
      world_size=world_size, 
      rank=rank)
  
  per_device_batch_size = int(argv.batch_size / ngpus_per_node)

  training_args = TrainingArguments(
      output_dir="/tmp/output/",
      num_train_epochs=argv.epochs, 
      per_device_train_batch_size=per_device_batch_size,
      per_device_eval_batch_size=per_device_batch_size,
      local_rank=local_rank,
  )

  def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}
  
  trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["test"],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
  )

  trainer.train()

  # Save the trained model locally
  model_filename = "pytorch-birt-model"
  local_path = os.path.join("/tmp", model_filename)
  trainer.save_model(local_path)

  if (os.path.exists(local_path)):
    # Upload the trained model to Cloud storage
    model_directory = argv.model_dir
    storage_path = os.path.join(model_directory, model_filename)
    blob = storage.blob.Blob.from_string(storage_path, client=storage.Client())

    files = [f for f in os.listdir(local_path) if os.path.isfile(os.path.join(local_path, f))]
    for file in files:
      local_file = os.path.join(local_path, file)
      blob.upload_from_filename(local_file)
  
    print(f"Saved model files in {model_directory}/{model_filename}")


if __name__ == "__main__":
    main()


创建一个源分发`dist/trainer-0.1.tar.gz`，并将带有训练应用程序的源分发上传到云存储桶，然后验证源分发是否存在于云存储桶中。

In [ ]:
! cd {PYTHON_PACKAGE_APPLICATION_DIR} && python3 setup.py sdist --formats=gztar

! gsutil cp {source_package_file_name} {python_package_gcs_uri}

! gsutil ls -l {python_package_gcs_uri}

在Vertex AI上使用Reduction Server运行自定义训练作业

配置一个自定义作业，使用预先构建的PyTorch容器映像和打包为Python源代码分发的训练代码。

In [ ]:
print(f"APP_NAME={APP_NAME}")
print(
    f"PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI={PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI}"
)
print(f"python_package_gcs_uri={python_package_gcs_uri}")
print(f"python_module_name={python_module_name}")

创建一个训练作业

您可以使用[Python的Vertex AI SDK](https://cloud.google.com/vertex-ai/docs/start/client-libraries#client_libraries)来创建一个自定义训练作业。

In [ ]:
JOB_NAME = f"pytorch-birt-reduction-server-{UUID}"
print(f"JOB_NAME={JOB_NAME}")

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=f"{JOB_NAME}",
    python_package_gcs_uri=python_package_gcs_uri,
    python_module_name=python_module_name,
    container_uri=PRE_BUILT_TRAINING_CONTAINER_IMAGE_URI,
)

#### 定义训练集群工作人员池和实验配置参数

Reduction Server可以与使用NVIDIA NCCL库进行全局归约集合运算的任何分布式训练框架一起使用。您无需更改或重新编译您的训练应用程序。

Google Cloud指南[分布式训练](https://cloud.google.com/vertex-ai/docs/training/distributed-training)提供了有关如何在Vertex AI上运行分布式训练作业的详细说明。

In [ ]:
# Training cluster worker pool configuration
REPLICA_COUNT = 3
MACHINE_TYPE = "n1-standard-16"
ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
ACCELERATOR_COUNT = 2

# Reduction Server configuration
REDUCTION_SERVER_COUNT = 4
REDUCTION_SERVER_MACHINE_TYPE = "n1-highcpu-16"
REDUCTION_SERVER_IMAGE_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest"
)
ENVIRONMENT_VARIABLES = {"NCCL_DEBUG": "INFO"}

# Training experiment parameters
EPOCHS = 2
BATCH_SIZE = 32
MODEL_DIR = f"{BUCKET_URI}/{JOB_NAME}"

training_args = [
    "--epochs",
    str(EPOCHS),
    "--batch_size",
    str(BATCH_SIZE),
    "--model_dir",
    MODEL_DIR,
]

提交培训任务

在定义了培训集群配置参数之后，使用Python的Vertex AI SDK提交和监控一个训练任务。

*注意：使用Python的Vertex AI SDK提交培训任务时，会创建一个训练管道，该管道会在Vertex AI训练服务上启动自定义任务。*

In [ ]:
model = job.run(
    replica_count=REPLICA_COUNT,
    machine_type=MACHINE_TYPE,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=ACCELERATOR_COUNT,
    reduction_server_replica_count=REDUCTION_SERVER_COUNT,
    reduction_server_machine_type=REDUCTION_SERVER_MACHINE_TYPE,
    reduction_server_container_uri=REDUCTION_SERVER_IMAGE_URI,
    environment_variables=ENVIRONMENT_VARIABLES,
    args=training_args,
    sync=True,
)

#### 监控培训工作（可选）

您可以通过以下链接监视从Cloud Console启动的自定义作业[这里](https://console.cloud.google.com/vertex-ai/training/training-pipelines/)，或使用gcloud CLI命令 [`gcloud beta ai custom-jobs stream-logs`](https://cloud.google.com/sdk/gcloud/reference/beta/ai/custom-jobs/stream-logs)。

验证模型工件（可选）

您可以在训练应用程序成功完成作业后，通过验证写入云存储桶的模型工件。

In [ ]:
print(f"Model artifacts are available at {MODEL_DIR}")

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
import os

delete_custom_job = True
delete_bucket = False

if delete_custom_job:
    try:
        job.delete()
    except Exception as e:
        print(e)

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI